In [1]:

from signal_database.labelSelectedData import SignalDB,SignalBundle,LabeledData
from bagfile_io.bagfile_reader import bagfile_reader,write_to_bagfile
import argparse
import numpy as np

import matplotlib.pyplot as plt
from plot_generator import plotResult_colorbars
import pandas as pd
from sklearn.utils.class_weight import compute_sample_weight
from time_series_learning import label_map,relabel_data,get_signal_features,LSTM_model,SeqGen,load_db,get_wavelet_features
from keras.callbacks import TensorBoard
from keras.models import load_model
import cPickle as pickle
from std_msgs.msg import String


/home/guru/Dropbox/lab-hp/workspace/research/end_to_end/end_to_end_env/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [18]:


__doc__ = "Adds bag files to the database. Used to create training data for detecting compliant actions "

bagfile = "../some unlabeled data/somepartgrab10.bag"


label_names_df = pickle.load(open("label_names_df.pkl", "rb"))

bfr = bagfile_reader(bagfile)



wrench1_,wrencht1 = bfr.get_topic_msgs("/ftmini40")
wrench2_,wrencht2 = bfr.get_topic_msgs("/ftmini402")
labels_,labelst = bfr.get_topic_msgs("/labels")

timesamples = list(wrencht1)

# labels = [label_messsage.data for label_messsage in labels_]

wrench1 = np.array([[f.wrench.force.x,f.wrench.force.y,f.wrench.force.z,
                f.wrench.torque.x,f.wrench.torque.y,f.wrench.torque.z] for f in wrench1_])

wrench2 = np.array([[f.wrench.force.x,f.wrench.force.y,f.wrench.force.z,
                f.wrench.torque.x,f.wrench.torque.y,f.wrench.torque.z] for f in wrench2_])


wrench1 = np.array([np.interp(timesamples,  wrencht1, wrench1[:,ii]) for ii in range(6)]).transpose()
wrench2 = np.array([np.interp(timesamples,  wrencht2, wrench2[:,ii]) for ii in range(6)]).transpose()


data = np.transpose(np.append(wrench1,wrench2,axis = 1)).tolist()

#addings stuff to the database
sb = SignalBundle(data,timesamples)
ld = LabeledData(sb)


In [19]:
# 
# labels = [label_str.data for label_str in labels_]
# 
# 
# if len(labels) == 0:
#     labels = np.shape(wrench1)[0]*['']

In [20]:
# ld.labels = labels
data = [ld]
timestamps = np.array(ld.signal_bundle.timestamps)



In [21]:

X_signal,Y = get_signal_features(data)
X_wavelet, _ = get_wavelet_features(data)
print np.shape(X_wavelet)



(38076, 4) (38076,)


(38076, 80)


In [12]:
print np.shape(X_wavelet),np.shape(X_signal),np.shape(data[0].signal_bundle.signals[0])

(381, 80) (38076, 4) (38076,)


In [ ]:

X = np.append(X_signal,X_wavelet,axis = 1)
#X = X_signal
# model_LSTM_saved = load_model("./trained_models/LSTM_only.hdf5")
model_LSTM_saved = load_model("./trained_models/LSTM_wavelet_10ep.hdf5")


window = 100
batch_stride = 1
batch_size = 1
num_classes = len(label_names_df)
data_dim = np.shape(X)[1]

model_LSTM = LSTM_model(data_dim, window, batch_size, num_classes, stateful=False)

model_LSTM.set_weights(model_LSTM_saved.get_weights())

skip = 100

Y_one_hot = pd.get_dummies(Y,columns=list(label_names_df))
sg = SeqGen(X[0:-1:skip],Y_one_hot[0:-1:skip],1,window,batch_stride=1)
sg_timesamples = SeqGen(X[0:-1:skip],timesamples[0:-1:skip],1,window,batch_stride=1)


(38076, 4) (381,)


In [8]:

predictions = np.array([]).reshape(0,len(label_names_df))
timesamples_predict = []

for ii in range(len(sg)):
    predictions = np.append(model_LSTM.predict(sg[ii][0],batch_size=batch_size),predictions,axis = 0)
    timesamples_predict = sg_timesamples[ii][1].tolist() + timesamples_predict

print np.shape(sg[ii][1])

predict_df = pd.DataFrame(predictions, columns=label_names_df)
predict_list = list(predict_df.idxmax(axis=1))



print predict_list




(1, 8)
['freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'grab', 'grab', 'grab', 'grab', 'grab', 'freespace', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', 'grab', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', 'hold', '', 'release', 'rele

In [49]:
plt.plot(timesamples_predict)

In [10]:
# plt.plot(timesamples_predict_zeroed)

NameError: name 'timesamples_predict_zeroed' is not defined

In [11]:
%matplotlib

Using matplotlib backend: TkAgg


In [62]:
indices = np.argsort(timesamples_predict)
timesamples_predict_ = np.sort(timesamples_predict)
indices =  np.argsort(timesamples_predict)
predict_list_sorted = [predict_list[indice] for indice in indices]
print predict_list_sorted

['still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'still', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', 'hand_on', '', '', '', '', '', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespace', 'freespac

In [57]:
indices = np.argsort(timesamples_predict)
timesamples_predict_ = np.sort(timesamples_predict)

test_one_hot_predict_series_ = [predict_list[indice] for indice in indices]

In [ ]:
test_one_hot_predict_series_

In [17]:
test_one_hot_predict_series = []
test_one_hot_truth_series = []


for ii in range(len(sg)):
    test_one_hot_predict = model_LSTM.predict(sg[ii][0], batch_size=batch_size)
    test_one_hot_predict_df = pd.DataFrame(test_one_hot_predict, columns=label_names_df)
    test_one_hot_predict_series_batch = list(test_one_hot_predict_df.idxmax(axis=1))
    test_one_hot_predict_series = test_one_hot_predict_series + test_one_hot_predict_series_batch

    # test_one_hot_truth = (sg[ii][1] > 0.9).astype(int)
    # print np.shape(test_one_hot_truth)
    # test_one_hot_truth_df = pd.DataFrame(test_one_hot_truth, columns=label_names_df)
    # test_one_hot_truth_series_batch = list(test_one_hot_truth_df.idxmax(axis=1))
    # test_one_hot_truth_series = test_one_hot_truth_series + test_one_hot_truth_series_batch

plt.figure(1)
ax = plt.subplot(2, 1, 1)

plotResult_colorbars(test_one_hot_predict_series_, range(len(test_one_hot_predict_series_)),
                     labelNames=list(label_names_df) + [''], ax=ax, medfiltwidth=1)
# ax.set_xlim([min(timesamples_predict), max(timesamples_predict)])
# ax = plt.subplot(2, 1, 2)
# 
# plotResult_colorbars(test_one_hot_truth_series, timesamples_predict,
#                      labelNames=list(label_names_df) + [''], medfiltwidth=1)
# ax.set_xlim([0, len(test_one_hot_truth_series)])


In [50]:
out_labels = np.array(pd.DataFrame(data = test_one_hot_predict_series_))
out_labels = np.repeat(out_labels,100)
timesamples_predict_interp = np.linspace(min(timesamples_predict_), max(timesamples_predict_),num = len(out_labels))
plt.figure(1)
ax = plt.subplot(2, 1, 1)
plotResult_colorbars(out_labels.tolist(), range(len(out_labels)),
                     labelNames=list(label_names_df) + [''], ax=ax, medfiltwidth=1)


In [19]:
plt.figure()
plt.plot(timesamples_predict_interp)


In [20]:
import rospy

# labels_t = [rospy.Time.from_sec(sec) for sec in timesamples_predict_interp]
labels_t = timesamples_predict_interp.tolist()

In [21]:
plt.figure()
plt.plot(labels_t)


'hold'

In [63]:
from copy import deepcopy

string_message_list = []

for label in predict_list_sorted:
    message = String()
    message.data = label
    string_message_list.append(message)

bigger_string_message_list = []

for message in string_message_list:
    bigger_string_message_list = bigger_string_message_list + [message]*100    


write_to_bagfile(bagfile, '/labels11',bigger_string_message_list, labels_t, 'a', createbackup=False)


In [41]:
import rosbag
topicname = '/labels8'
with rosbag.Bag(bagfile, 'a') as bag:
    for msg,stamp in zip(string_message_list,labels_t):
        bag.write(topicname, msg, rospy.Time.from_sec(stamp))

In [42]:
bfr = bagfile_reader(bagfile)

In [43]:
rospy.Time.from_sec(labels_t[-1]).to_sec()

1525906423.6802917

In [44]:
_,timestamps_check = bfr.get_topic_msgs('/labels8')

In [45]:
print labels_t[-1],timestamps_check[-1]

1525906423.68 1525906423.6802917


In [38]:
len(string_message_list)

381